# Математическая статистика
## Практикум 2


**Правила:**

* Дедлайн ** 5 июня 2022 в 24:00**. После дедлайна работы не принимаются, кроме случаев наличия уважительной причины.
* Выполненную работу нужно отправить на почту MarkashevaVA@mgppu.ru и также на w9071981@yandex.ru, указав тему письма `"Практикум 1[номер группы] Фамилия Имя"`. Квадратные скобки обязательны.
* Прислать нужно ноутбук и его pdf-версию (без архивов). Названия файлов должны быть такими: `N.ipynb` и `N.pdf`, где `N` - ваш  номер в журнале группы и ФИО(нумерация начинается с 01,02,03... и т.д.)
* Решения, размещенные на каких-либо интернет-ресурсах не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлению возможности списать.
* Для выполнения задания используйте этот ноутбук в качествие основы, ничего не удаляя из него.
* Никакой код из данного задания при проверке запускаться не будет.

*НЕЛЬЗЯ ИСПОЛЬЗОВАТЬ ЦИКЛЫ, кроме самых отчаянных ситуаций.

### Номер ВАШЕГО ВАРИАНТА есть номер вашей фамилии в списке группы. В дальнейшем будем обозначать его $N$



In [1022]:
import numpy as np
import scipy.stats as sps
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd


**Задача.1**  Пусть $x_1,\ldots,x_n$ — реализация из модели сдвига показательного закона
	с плотностью распределения 
	$$
		f_\theta(u) = e^{-\lambda(u-\theta)}\cdot I_{\{u\geq\theta\}} 
		= 
		\begin{cases}
		 	e^{-\lambda(u-\theta)}, & u\geq\theta,\\
			0, & u<\theta.
		\end{cases}
	$$	
Оценить $\theta$ с помощью метода моментов и метода максимального правдоподобия. Реализуйте эту задачу в Python:

* сгенерируйте $\theta$ из равномерного распределения на  $[1,N]$ и возьмите $\lambda=|N-10|/N$, где `N` - ваш  номер в журнале группы и ФИО(нумерация начинается с 01,02,03... и т.д.)
* сгенерируйте выборку из распределения с плотностью $f_\theta(u)$ размера  $n=1\,0000$; 
* постройте алгоритм нахождения оценки методом моментов и методом правдоподобия (мы такой пример решали на занятии), а потом найдите значения оценок метода моментов и метода максимального правдоподобия, которые генерируются автоматически Питоном.

### Задача 1.

**Я имею 5 номер в списке учащихся**

In [1023]:
theta = np.random.uniform(1, 5) # генерирую Тета из равномерного распределения
lambd = abs(5-10)/5 # параметр лямбда

# в плотности распределения видно, что Тета отвечает за отрицательное смещение функции на графике
# исходя из этого, генерирую выборку
sample = np.random.exponential(lambd, size = 10000) - theta # размер выборки 10000

# оценки методов
esimate_mme = np.mean(sample) - lambd  # метод моментов
esimate_mle = np.min(sample) # метод максимального правдоподобия

print("Оценка метода моментов: ", esimate_mme)
print("Оценка метода максимального правдоподобия: ", esimate_mle)

Оценка метода моментов:  -3.4839356715246126
Оценка метода максимального правдоподобия:  -3.505551096169506


Значения при применении разных методов расходятся лишь в тысячных, что говорит о том, что они работают.
Что касается поисков оценок методов "вручную" - с этим у меня возникли некоторые сложности, а именно с нахождением интеграла 
для получения матожидания заданной плотности распределения.

(я также пробовал создать пользовательское распределение с помощью библиотеки scipy, но попытка не обвенчалась успехом)

### Задача 2. Критерий согласия 

In [1024]:
sps.kstest(sample, 'expon') 

KstestResult(statistic=0.9667, pvalue=0.0)

Оценив параметр **statistic** можно отвергать гипотезу о том, что представленная выборка, осложненная параметром Тета,
соответствует экспоненциальному распределнию. Уровень значимости (pvalue) равен нулю. 
Ниже приведу пример того, как ведет себя функция kstest, если убрать Тету.

In [1032]:
sample = np.random.exponential(lambd, size = 10000)
sps.kstest(sample, 'expon') 

KstestResult(statistic=0.005417273830428115, pvalue=0.929307389731466)

Здесь же видно близкий к нулю **statistic** и почти равный единице **pvalue**, что позволяет нам принять гипотезу.

### Задача 3. Доверительные интервалы

In [1041]:
theta = np.random.uniform(-5,5) # Тета из равномерного распределения
sample = np.random.normal(theta, 4, size = 100) # выборка, размер 100
alpha = sps.kstest(sample, 'norm').pvalue # берем уровень значимостти из критерия Колмогорова
sigma = np.std(sample) # среднеквадратическое отклонение для выборки

**Доверительный интервал: теоретический**

In [1043]:
print("Доверительный интервал: ", 1 - alpha)

Доверительный интервал:  0.9999999999996069


**Доверительный интервал: параметрический бутстрэп**

In [1019]:
number_of_bootstrap_samples = 5 # количество выборок
size_of_bootstrap_samples = 100 # размер выборок

mean = np.mean(sample) # оценка параметра theta 
bootstrap_samples = np.random.normal(mean,sigma,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) # выборка
bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples) # среднее для каждой выборки 

# Вычисляем параметрический бутстрэп доверительный интервал
CI_Bootstrap_Parametric = [np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2)]

print("Интервал параметрического бутстрэпа: ",CI_Bootstrap_Parametric)

Интервал параметрического бутстрэпа:  [2.7153216877954924, 3.1008150001793044]


**Доверительный интервал: непараметрический бутстрэп**

In [1018]:
bootstrap_samples = np.random.choice(sample,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) # выборка
bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples) # среднее для каждой выборки 

# Вычисляем непараметрический бутстрэп доверительный интервал
CI_Bootstrap_Nonparametric = [np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2)] 

print("Интервал непараметрического бутстрэпа: ",CI_Bootstrap_Nonparametric)

Интервал непараметрического бутстрэпа:  [2.9711623639799596, 3.4992121138782837]


# Вывод

Опыт показал, что проще (и точнее) использовать предлагаемые Питоном фукнции для построения выборок и анализа параметров различными методами. 
Говоря о доверительных интервалах бутстрэпа:
в моем случае интервалы накладываются друг на друга и, кажется, можно сказать, что отсутствуют значимые отличия и мы не можем отвергнуть нулевую гипотезу.

Ниже представлю отрывок неудавшейся программы по реализации методов вручную, значения там слишком расходились.

In [1074]:
import numpy as np
import math

lambd = 1
theta = np.random.uniform(1, 5)
u = np.linspace(theta,sps.expon.ppf(0.99), 100)
sample = -np.exp(-lambd*(u-theta)) # здесь я пытался получить первообразную из функции плотности - функцию распределения.
esimate_mme = np.mean(sample) - lambd 
esimate_mle = np.min(sample)
print(f'Оценка метода моментов: {esimate_mme}\nОценка метода максимального праводоподобия: {esimate_mle}')

Оценка метода моментов: -1.4343152537908652
Оценка метода максимального праводоподобия: -1.0


In [2]:
np.random.seed(100) # зафиксируем seed

### Seed

Для того, чтобы каждый раз при использовании генератора псевдослучайных чисел получать идентичные последовательности, используется функция set.seed (от set - задать, установить, и seed - начальное число). Как следует из названия, эта функция фиксирует число, служащее начальной точкой для запуска алгоритма генерации псевдослучайных чисел. В качестве аргумента функции указывают любое целое число (не важно, какое именно). 


* **np.random.seed(<число>)** —
настраивает генератор случайных чисел на новую последовательность. Если данная функция в программе не вызывается, по умолчанию используется системное время. 

В качестве примера приведем еще фукции для распределений, которые мы упоминали в курсе по теории вероятностей:
* **np.random.binomial(n, p[, size])** — биномиальное распределение  
* **np.random.exponential([scale, size])** — экспоненциальное распределение  
* **np.random.poisson([lam, size])** — распределение Пуассона  
* **np.random.standard_cauchy([size])** — распределение Коши  

Обратите внимание на то, что в np.random.exponential параметр scale = 1/$\lambda$.

In [ ]:
Пример.

In [378]:
n = 1000 # размер выборки

scale = 5 # параметр показательного распределения (scale)
drift = np.random.uniform(1,10) # а это сдвиг, который нам неизвестен и который мы будем оценивать

In [452]:
sample = drift+np.random.exponential(scale, size = n) # генерируем выборку размера n

In [380]:
esimate_mme = np.mean(sample) - scale  # оценка метода моментов
esimate_mle = np.min(sample) # оценка метода максимального правдоподобия

In [383]:
print("Истинное значение параметра: ", drift)
print("Оценка метода моментов: ", esimate_mme)
print("Оценка метода максимального правдоподобия: ", esimate_mle)

Истинное значение параметра:  1.2107897360187567
Оценка метода моментов:  1.201419863609456
Оценка метода максимального правдоподобия:  1.2109236264358023


**Задача.2**  Мы нашли точечные оценки методом моментов и методом правдоподобия, теперь надо оценить используя соответствующий параметрическому случаю критерий согласия, согласуется ли полученные распределения с данной выборкой.

**Задача 3.** Сравнить на выборках размера 100 для $\mathcal{N}(\theta,4)$ доверительные интервалы:
(1) теоретический, (2) на основе параметрического бутстрэпа, (3) на основе непараметрического бутстрэпа. Сам параметр $\theta$ сгенерировать из равномерного распределения на $[-5,5]$. 

In [ ]:
# Фиксируем параметры задачи

n = ?? # размер выборки 
alpha = ?? #уровень значимости

theta = np.random.uniform(-5,5) # неизвестное среднее нормального распределения
sigma = ?? # известная sigma нормального распределения

In [ ]:
# Сгенерируем выборку из нужного распределения
sample = ??

In [35]:
theta = np.random.uniform(-5,5)
print("Значение theta равно",theta)

Значение theta равно 4.1006187540576775


### Теоретический доверительный интервал

Напомним, что теоретический доверительный интервал вычисляется следующим образом: 

$$
\mathbb{P}\left( \bar{X} - \frac{c_{1-\alpha/2}\sigma}{\sqrt{n}} < \mu < \bar{X} + \frac{c_{1-\alpha/2}\sigma}{\sqrt{n}} \right) = 1-\alpha,
$$
где $c_{\alpha}$ — квантиль распределения $\mathcal{N}(0,1)$ уровня $\alpha$.

 Вычисляем теоретический доверительный интервал используя функции $stats.norm.ppf(1-alpha/2)$ для вычисления кваниля $c_{\alpha}$.



### Доверительный интервал на основе параметрического бутстрэпа

In [5]:
# Параметры для бутстрэпа
number_of_bootstrap_samples = 10 # количество бутстрэп-выборок
size_of_bootstrap_samples = 20 # размер бутстрэп-выборок

In [ ]:
# Оцениваем неизвестный параметр theta 
mean = np.mean(sample) 

In [ ]:
# Генерируем выборку из распределения N(sample_mean, sigma)
bootstrap_samples = np.random.normal(mean,sigma,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) 

In [ ]:
# Считаем среднее для каждой выборки 
bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples)

In [ ]:
# Вычисляем параметрический бутстрэп доверительный интервал
CI_Bootstrap_Parametric = [np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2)]

### Доверительный интервал на основе непараметрического бутстрэпа

In [ ]:
# Будем использовать те же параметры
number_of_bootstrap_samples = 10 # количество бутстрэп-выборок
size_of_bootstrap_samples = 20 # размер бутстрэп-выборок

In [ ]:
# Генерируем выборку из распределения N(bootstrap_mean, sigma)
bootstrap_samples = np.random.choice(sample,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) 

In [ ]:
# Считаем среднее для каждой выборки 
bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples)

In [ ]:
# Вычисляем непараметрический бутстрэп доверительный интервал
CI_Bootstrap_Nonparametric = [np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2)]

In [6]:
### Сделайте выводы: как сравнить полученные доверительные интервалы? как сравнить методы? какие лучше и почему?